# **🚗 Dynamic Pricing for Urban Parking Lots**
# **Capstone Project – Summer Analytics 2025**
**Built with NumPy, Pandas, Bokeh & Pathway**


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Install Required Packages
!pip install -q bokeh pathway

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
from math import radians, sin, cos, sqrt, atan2
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import Legend
from bokeh.io import push_notebook
output_notebook()

# **Load Dataset**

In [3]:
# 📁 Load Dataset
df = pd.read_csv("/content/dataset (1).csv")

In [5]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [6]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

# **Preprocessing Function**

In [7]:
# 🧹 Data Preprocessing
def preprocess_data(df):
    df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
    df.rename(columns={
        'SystemCodeNumber': 'Location',
        'Latitude': 'Lat',
        'Longitude': 'Lon',
        'IsSpecialDay': 'SpecialDay',
        'QueueLength': 'QueueLength',
        'TrafficConditionNearby': 'Traffic',
    }, inplace=True)
    df['Traffic'] = df['Traffic'].map({'low': 0.3, 'medium': 0.6, 'high': 1.0})
    df['VehicleType'] = df['VehicleType'].astype(str).str.lower()
    df['SpecialDay'] = df['SpecialDay'].astype(int)
    return df

df = preprocess_data(df)

# **Model 1: Linear Pricing**

In [8]:
def model1_linear(df, alpha=0.05):
    df['Price_Model1'] = 10 + alpha * (df['Occupancy'] / df['Capacity']) * 10
    return df

# **Model 2: Demand-Based Pricing**

In [9]:
def model2_demand(df, alpha=0.5, beta=0.3, gamma=0.2, delta=0.4, epsilon=0.3):
    weights = {'car': 1, 'bike': 0.5, 'truck': 1.5}
    df['VehicleWeight'] = df['VehicleType'].map(weights).fillna(1.0)
    demand = (alpha * df['Occupancy'] / df['Capacity'] +
              beta * df['QueueLength'] -
              gamma * df['Traffic'] +
              delta * df['SpecialDay'] +
              epsilon * df['VehicleWeight'])
    norm_demand = (demand - demand.min()) / (demand.max() - demand.min())
    df['Price_Model2'] = 10 * (1 + 1.5 * norm_demand)
    df['Price_Model2'] = df['Price_Model2'].clip(5, 20)
    df['NormalizedDemand'] = norm_demand
    return df

# **Haversine Distance**

In [10]:
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# **Model 3: Competitive Pricing**

In [11]:
def model3_competition(df):
    df['Price_Model3'] = df['Price_Model2'].copy()
    timestamps = df['timestamp'].unique()
    for ts in timestamps:
        subset = df[df['timestamp'] == ts]
        coords = subset[['Lat', 'Lon']].values
        for i, (idx, row) in enumerate(subset.iterrows()):
            dists = haversine_np(row['Lat'], row['Lon'], coords[:,0], coords[:,1])
            dists[i] = np.inf  # ignore self
            nearest_idx = np.argmin(dists)
            comp_price = subset.iloc[nearest_idx]['Price_Model2']
            own_price = row['Price_Model2']
            occupancy = row['Occupancy']
            capacity = row['Capacity']
            new_price = own_price
            if occupancy >= capacity:
                new_price *= 0.9  # encourage rerouting
            elif comp_price > own_price:
                new_price *= 1.1
            df.loc[idx, 'Price_Model3'] = np.clip(new_price, 5, 25)
    return df

# **Rerouting Flag**

In [12]:
def add_rerouting_flag(df):
    df['RerouteSuggested'] = False
    timestamps = df['timestamp'].unique()
    for ts in timestamps:
        subset = df[df['timestamp'] == ts]
        coords = subset[['Lat', 'Lon']].values
        for i, (idx, row) in enumerate(subset.iterrows()):
            if row['Occupancy'] < row['Capacity']:
                continue
            dists = haversine_np(row['Lat'], row['Lon'], coords[:,0], coords[:,1])
            dists[i] = np.inf
            nearest_idx = np.argmin(dists)
            competitor_price = subset.iloc[nearest_idx]['Price_Model2']
            if competitor_price < row['Price_Model2']:
                df.at[idx, 'RerouteSuggested'] = True
    return df

# **Apply All Models**

In [13]:
df = model1_linear(df)
df = model2_demand(df)
df = model3_competition(df)
df = add_rerouting_flag(df)

# **Result's**

In [14]:
df[['Price_Model1', 'Price_Model2', 'Price_Model3', 'RerouteSuggested']]

,Price_Model1,Price_Model2,Price_Model3,RerouteSuggested
0,10.052860,11.384463,12.522909,False
1,10.055459,11.391759,11.391759,False
2,10.069324,12.272582,13.499840,False
3,10.092721,12.338242,13.572066,False
4,10.129983,12.021856,13.224042,False
...,...,...,...,...
18363,10.395052,NaN,NaN,False
18364,10.387240,14.006686,14.006686,False
18365,10.372917,13.966490,15.363139,False
18366,10.344010,13.043455,14.347801,False


# **Visulisation**

In [15]:
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Legend
from bokeh.io import push_notebook

def bokeh_plot_all(df):
    plots = []
    locations = df['Location'].unique()

    for lot in locations:
        df_lot = df[df['Location'] == lot].sort_values('timestamp')
        if df_lot.empty:
            continue
        source = ColumnDataSource(df_lot)
        p = figure(x_axis_type="datetime", title=f"📍 Real-Time Pricing for {lot}", width=750, height=300)
        p.line(x='timestamp', y='Price_Model1', source=source, line_color="blue", legend_label="Model 1")
        p.line(x='timestamp', y='Price_Model2', source=source, line_color="green", legend_label="Model 2")
        p.line(x='timestamp', y='Price_Model3', source=source, line_color="red", legend_label="Model 3")
        p.xaxis.axis_label = "Timestamp"
        p.yaxis.axis_label = "Price ($)"
        p.legend.location = "top_left"
        p.legend.click_policy = "hide"
        plots.append(p)
    show(column(*plots))

bokeh_plot_all(df)

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [16]:
# ⚙️ (OPTIONAL) Streaming Logic Placeholder (Pathway)

# STREAMING SETUP (Pathway)
import pathway as pw
from datetime import datetime # Keep datetime imported for other uses if needed

# Define the schema
class ParkingEvent(pw.Schema):
    timestamp: str # Changed to str to align with working schema example
    Location: str
    Lat: float
    Lon: float
    Capacity: int
    Occupancy: int
    QueueLength: int
    Traffic: float
    SpecialDay: int
    VehicleType: str

# Simulate Streaming Data
# Removed .with_delay(0.5) as it was causing an AttributeError
stream = pw.io.csv.read("dataset (1).csv", schema=ParkingEvent)

# Define pricing transformation using your logic here
# Example: stream = stream.select(pw.this.timestamp, pw.this.Occupancy / pw.this.Capacity)
# Replacing the placeholder '...' with a minimal select to make the cell runnable
stream = stream.select(pw.this.timestamp, pw.this.Location)

In [17]:
print(stream.schema)

id          | timestamp | Location
ANY_POINTER | STR       | STR     


In [18]:
df_enriched = df[[
    "timestamp", "Location", "Occupancy", "Capacity", "QueueLength",
    "Traffic", "SpecialDay", "VehicleType", "Price_Model1", "Price_Model2", "Price_Model3"
]].copy()
df_enriched.rename(columns={"timestamp": "Timestamp"}, inplace=True)
df_enriched.to_csv("parking_stream_enriched.csv", index=False)


In [19]:
# 🧾 Step 1: Define Enriched Schema
class EnrichedParkingSchema(pw.Schema):
    Timestamp: str
    Location: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: float
    SpecialDay: int
    VehicleType: str
    Price_Model1: float
    Price_Model2: float
    Price_Model3: float


In [20]:
# 🛰 Step 2: Load the enriched CSV stream
enriched_stream = pw.io.csv.read("parking_stream_enriched.csv", schema=EnrichedParkingSchema)
print(enriched_stream.schema)

id          | Timestamp | Location | Occupancy | Capacity | QueueLength | Traffic | SpecialDay | VehicleType | Price_Model1 | Price_Model2 | Price_Model3
ANY_POINTER | STR       | STR      | INT       | INT      | INT         | FLOAT   | INT        | STR         | FLOAT        | FLOAT        | FLOAT       


In [21]:

# 🧠 Step 3: Example derived logic – Adjust final price based on congestion & vehicle type
@pw.udf
def final_price(price: float, traffic: float, veh_type: str) -> float:
    traffic_penalty = traffic * 0.1
    vehicle_weight = {"car": 1.0, "bike": 0.5, "truck": 1.5}.get(veh_type.lower(), 1.0)
    adjusted = price + (price * traffic_penalty * vehicle_weight)
    return round(min(max(adjusted, 5), 25), 2)

In [22]:
# ⚙️ Step 4: Enrich stream with final adjusted price
enriched_output = enriched_stream.select(
    Timestamp=enriched_stream.Timestamp,
    Location=enriched_stream.Location,
    Occupancy=enriched_stream.Occupancy,
    Capacity=enriched_stream.Capacity,
    QueueLength=enriched_stream.QueueLength,
    Traffic=enriched_stream.Traffic,
    SpecialDay=enriched_stream.SpecialDay,
    VehicleType=enriched_stream.VehicleType,
    Price_Model1=enriched_stream.Price_Model1,
    Price_Model2=enriched_stream.Price_Model2,
    Price_Model3=enriched_stream.Price_Model3,
    FinalPrice=final_price(enriched_stream.Price_Model3, enriched_stream.Traffic, enriched_stream.VehicleType)
)

In [23]:
enriched_output

<pathway.Table schema={'Timestamp': <class 'str'>, 'Location': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>, 'QueueLength': <class 'int'>, 'Traffic': <class 'float'>, 'SpecialDay': <class 'int'>, 'VehicleType': <class 'str'>, 'Price_Model1': <class 'float'>, 'Price_Model2': <class 'float'>, 'Price_Model3': <class 'float'>, 'FinalPrice': <class 'float'>}>

In [24]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = enriched_stream.with_columns(
    t = enriched_stream.Timestamp.dt.strptime(fmt),
    day = enriched_stream.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [25]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Calculate a simple price based on occupancy for demonstration
delta_window = data_with_time.with_columns(
    price = 10 + 0.05 * (pw.this.Occupancy / pw.this.Capacity) * 10
)

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [26]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

ERROR:pathway_engine.connectors:Parse error: failed to parse value "" at field "Traffic" according to the type float in schema: cannot parse float from empty string
ERROR:pathway_engine.connectors:Parse error: failed to parse value "" at field "Traffic" according to the type float in schema: cannot parse float from empty string
ERROR:pathway_engine.connectors:Parse error: failed to parse value "" at field "Traffic" according to the type float in schema: cannot parse float from empty string
ERROR:pathway_engine.connectors:Parse error: failed to parse value "" at field "Traffic" according to the type float in schema: cannot parse float from empty string
ERROR:pathway_engine.connectors:Parse error: failed to parse value "" at field "Traffic" according to the type float in schema: cannot parse float from empty string
ERROR:pathway_engine.connectors:Parse error: failed to parse value "" at field "Traffic" according to the type float in schema: cannot parse float from empty string
ERROR:path

KeyboardInterrupt: 